Developing the project especified in the proposal

- Layer 1: input layer - Dimensions: D (number of sensors or channels) X 400 (number of samples)
    - for dataset 1: D=10; for dataset 2 and 3: D=12
- Layer 2: Convolutional layer 64
- Layer 3: Convolutional layer 64
- Layer 4: Convolutional layer 64
- Layer 5: Convolutional layer 64
- Droupout
- Layer 6: Dense layer LSTM 128
- Dropout
- Layer 7: Dense layer LSTM 64
- Layer 8: Softmax layer 50 classes
- RMSProp update rule
- mini batch gradient descent - size=100, learning rate=0.001, decay factor=0.9
- dropout: p=0.5
- Test set: 1/4
- Training set: 3/4

In [42]:
# All three datasets already downloaded to ./datasets/db1, ./datasets/db2 and ./datasets/db3, respectively
import nina_helper as nh
import numpy as np

DB1_PATH = "./datasets/db1"
DB2_PATH = "./datasets/db2"
DB3_PATH = "./datasets/db3"

# Decide window length (200ms window, 100ms increment)
window_len = 20
window_step = 1

# Choose subject and get info
subject = 1
info_dict = nh.db1_info()  # Get info

# Get EMG, repetition and movement data, don't cap maximum length of rest
data_dict = nh.import_db1(DB1_PATH, subject)

# Create a balanced test - training split based on repetition number
reps = info_dict['rep_labels']
nb_test_reps = 3
train_reps, test_reps = nh.gen_split_balanced(reps, nb_test_reps)

# Normalise EMG data based on training set
emg_data = nh.normalise_emg(data_dict['emg'], data_dict['rep'], train_reps[0, :])

# Window data: x_all data is 4D tensor [observation, time_step, channel, 1] for use with Keras
# y_all: movement label, length: number of windows
# r_all: repetition label, length: number of windows
moves = np.array([1, 3, 5, 20])
x_all, y_all, r_all = nh.get_windows(reps, window_len, window_step,
                                  emg_data, data_dict['move'],
                                  data_dict['rep'],
                                  which_moves=moves)

train_idx = nh.get_idxs(r_all, train_reps[0, :])
train_data = x_all[train_idx, :, :, :]
one_hot_categorical = nh.to_categorical(y_all)

In [43]:
print (one_hot_categorical[0])
print (len(y_all))
print (r_all[0:-1:50])
print (max(y_all))
print (info_dict.keys())
print (info_dict['move_labels'])
print (info_dict['rep_labels'])

[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
14778
[ 1  1  1  1  1  1  1  2  2  2  2  2  2  2  3  3  3  3  3  3  3  4  4  4
  4  4  4  4  5  5  5  5  5  5  5  5  5  6  6  6  6  6  6  6  7  7  7  7
  7  7  7  7  7  8  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9  9 10 10
 10 10 10 10 10  1  1  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2  3
  3  3  3  3  3  3  3  3  3  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5
  5  5  5  6  6  6  6  6  6  6  6  6  6  7  7  7  7  8  8  8  8  8  8  8
  9  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10  1  1  1  1  1  1  1  1
  1  2  2  2  2  2  2  2  2  2  3  3  3  3  3  3  3  4  4  4  4  4  4  4
  4  4  5  5  5  5  5  5  5  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7
  7  7  8  8  8  8  8  8  8  9  9  9  9  9  9 10 10 10 10 10 10 10 10  1
  1  1  1  1  1  2  2  2  2  2  2  3  3  3  3  3  4  4  4  4  5  5  5  5
  5  5  5  6  6  6  6  6  6  7  7  7  7  7  7  7  8  8  8  8  8  9  9  9
  9  9 10 10 10 10 10 10]
20
dict_keys(['right_handed

In [9]:
one_hot_categorical

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])